In [29]:
import torch 
from transformers import ViTFeatureExtractor, ViTForImageClassification
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np 
from PIL import Image
import pandas as pd
import os 
# import wandb
from tqdm import tqdm
from functools import partial
classes = os.listdir("../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train")
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

In [30]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [31]:
train_dir = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train"
test_dir = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test"

In [32]:
class Standford_Car_Dataset(Dataset):
    
    def __init__(self, root, feature_extractor, labels):
        self.feature_extractor = feature_extractor 
        self.files = []
        self.labels = labels
        
        subdirs = sorted(os.listdir(root))
        
        for subdir in subdirs:
            for path, _, files in os.walk(os.path.join(root, subdir)):
                for file in files:
                    self.files.append((os.path.join(root, subdir, file), subdir))
    
    def __getitem__(self, index):
        img, label = self.files[index % len(self.files)]
        img = Image.open(img).convert("RGB")
        img = self.feature_extractor(img, return_tensors="pt")
        img["labels"] = torch.tensor(self.labels.index(label))
        return img
    
    def __len__(self):
        return len(self.files)

In [33]:
train_dataset = Standford_Car_Dataset(train_dir, feature_extractor, classes)
test_dataset = Standford_Car_Dataset(test_dir, feature_extractor, classes)
test_dataset, val_dataset = torch.utils.data.random_split(test_dataset, [6041, 2000])

In [34]:
batch_size = 32
num_workers = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [35]:
batch = next(iter(train_loader))
batch_images = batch["pixel_values"]
batch_labels = batch["labels"]

In [36]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
num_epochs = 10
lr = 2e-4
eval_steps = 100
record_steps= 10
save_checkpoint = 5

model_V = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=len(classes),
                                                   id2label={str(i): c for i, c in enumerate(classes)},
                                                   label2id={c: str(i) for i, c in enumerate(classes)},
                                                   ignore_mismatched_sizes=True).to(device)

optim = torch.optim.AdamW(model_V.parameters(), lr=lr)

training_stats_step = 10

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([196, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([196]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
from torch.optim.lr_scheduler import StepLR

scheduler = StepLR(optim, step_size=1, gamma=0.6)

In [38]:
def evaluate(model, eval_loader, eval_dataset):
    correct = 0
    eval_loss = []
    
    model.eval()
    for batch in tqdm(eval_loader):
        # extracting images and labels from batch 
        batch_images = batch["pixel_values"].squeeze(1).to(device)
        batch_labels = batch["labels"].to(device)
        
        # not training the model
        with torch.no_grad():
            outputs = model(pixel_values=batch_images, labels=batch_labels)
            loss = outputs[0]
            eval_loss.append(loss.item())
            correct += (torch.argmax(outputs["logits"], dim=1) == batch_labels).sum().item()
    
    # return eval accuracy and loss
    accuracy = (100 * correct / len(eval_dataset))
    avg_loss = (sum(eval_loss) / len(epoch_loss))
    
    return accuracy, avg_loss

In [39]:
tqdm = partial(tqdm, position=0, leave=True)

In [40]:
for epoch in range(1, num_epochs+1):
    # storing loss and accuracy across the epoch
    epoch_loss = []
    epoch_acc = []
    
    print(f"Epoch {epoch}")
    for index, batch in enumerate(tqdm(train_loader)):
        model_V.train()
        optim.zero_grad()

        # extract images and labels from batch
        batch_images = batch["pixel_values"].squeeze(1).to(device)
        batch_labels = batch["labels"].to(device)
        size = len(batch_images)
        
        outputs = model_V(pixel_values=batch_images, labels=batch_labels)
        
        loss = outputs[0]
        epoch_loss.append(loss.item())
        loss.backward()
        optim.step()
        
        correct = (torch.argmax(outputs["logits"], dim=1) == batch_labels).sum().item()
        acc = (100 * correct) / size
        epoch_acc.append(acc)
        
        # log the training metrics
        if index % record_steps == 0:
            print({'loss': loss, "acc" : acc})
        
    
    # calculate summary stats for each epoch 
    avg_accuracy = (sum(epoch_acc) / len(epoch_acc))
    avg_loss = (sum(epoch_loss) / len(epoch_loss))
    
    # we decay the loss over time 
    scheduler.step()
    
    # save checkpoints using torchscript 
    if epoch % save_checkpoint == 0:
        model_V.save_pretrained(f"Epoch {epoch}")
    
    # finding validation accuracy and loss
    val_acc, val_loss = evaluate(model_V, val_loader, val_dataset)
    print({'validation accuracy': val_acc, "validation loss" : val_loss})
    
    # summary stats at the end of the episode
    print("evaluating on validation set")
    print(f"val loss: {round(val_loss, 4)}, val acc: {round(val_acc, 4)}%")
    print(f"Epoch: {epoch}  avg loss: {round(avg_loss, 4)} avg acc: {round(avg_accuracy, 4)}%")

Epoch 1


  0%|          | 1/255 [00:01<07:54,  1.87s/it]

{'loss': tensor(5.3355, device='cuda:0', grad_fn=<NllLossBackward0>), 'acc': 0.0}


  4%|▍         | 11/255 [00:12<04:23,  1.08s/it]

{'loss': tensor(5.6026, device='cuda:0', grad_fn=<NllLossBackward0>), 'acc': 0.0}


...

{'loss': tensor(0.0259, device='cuda:0', grad_fn=<NllLossBackward0>), 'acc': 100.0}


 98%|█████████▊| 251/255 [04:34<00:04,  1.10s/it]

{'loss': tensor(0.0277, device='cuda:0', grad_fn=<NllLossBackward0>), 'acc': 100.0}


100%|██████████| 63/63 [00:31<00:00,  2.03it/s]

{'validation accuracy': 84.05, 'validation loss': 0.16716072594418246}
evaluating on validation set
val loss: 0.1672, val acc: 84.05%
Epoch: 10  avg loss: 0.0249 avg acc: 99.7794%


In [41]:
torch.save(model_V.state_dict(), "./trained_ViTForImageClassification_new.pt")
torch.save(model_V.state_dict(), 'ViTForImageClassification_new.h5')